In [1]:
import os
import numpy as np
import pandas as pd
import absl.logging
from nlp_embedding import GloVe, SmallBert, Bert, Word2vec, LargeBert
from nlp_classifier import NaiveBayes, SVM, XGBoost, CNN, BinaryCNN
from sklearn import preprocessing, metrics
from ast import literal_eval
absl.logging.set_verbosity(absl.logging.ERROR)

In [2]:
max_words = 400
dataset_name = 'small'
cnn_epochs = 5
indiv_genre = 'Rock'
optimizer = 'adam'

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
tf.get_logger().setLevel('ERROR')

class CNN2Step():
    def __init__(self, vec_len, class_count, optimizer, indiv_class):
        self.name = '2_step_cnn'
        self.model1 = BinaryCNN(vec_len, optimizer)
        self.model2 = CNN(vec_len, class_count - 1, optimizer)
        self.indiv_class = indiv_class

    def partial_fit(self, X, Y):
        Y_binary = np.array(Y == self.indiv_class).astype(int)
        self.model1.partial_fit(X.reshape(*X.shape, 1), Y_binary.reshape(-1, 1))
        
        X_other = X[Y != self.indiv_class]
        Y_other = Y[Y != self.indiv_class]
        self.model2.partial_fit(X_other.reshape(*X_other.shape, 1), Y_other.reshape(-1, 1))

    def predict(self, X):
        pred1 = self.model1.predict(X.reshape(*X.shape, 1))
        X2 = X[pred1 == 0]
        pred2 = self.model2.predict(X2.reshape(*X2.shape, 1))
        
        pred = np.zeros(len(X), dtype=int)
        pred[pred1 == 1] = self.indiv_class
        pred[pred1 == 0] = pred2

        print(np.unique(pred))
        print(np.unique(pred1))
        print(np.unique(pred2))
        
        return pred.flatten()

    def predict_proba(self, X):
        # we are not using it anyway for now
        pass
    
    def save(self, filename):
        self.model1.save(f'{filename}1')
        self.model2.save(f'{filename}2')

    def load(self, filename):
        self.model1 = models.load_model(f'{filename}1')
        self.model2 = models.load_model(f'{filename}2')

In [4]:
def train(data_x, data_y, nlp_embedding, nlp_classifier, label_encoder, batch_size, dataset_name, epochs=1, model_dir='models', start_idx=0, fname_end=''):
    print('Training...')
    fname = os.path.join(model_dir, dataset_name, f'model_{nlp_embedding.name}_{nlp_classifier.name}{fname_end}')
    data_y_enc = label_encoder.transform(data_y)
    
    for epoch in range(epochs):
        print(f'Epoch: {str(epoch + 1)}/{str(epochs)}')
        for i in range(start_idx, data_x.shape[0], batch_size):
            
            if i + batch_size > data_x.shape[0]:
                j = data_x.shape[0]
            else:
                j = i + batch_size
            
            print(f'Processing rows: {i} - {j - 1}')

            embeddings = nlp_embedding.embed_lyrics(data_x[i:j])
            nlp_classifier.partial_fit(embeddings, data_y_enc[i:j])
            nlp_classifier.save(fname)
        start_idx = 0
    
    print('Success!')

In [5]:
def test(data_x, nlp_embedding, nlp_classifier, label_encoder, batch_size, dataset_name, pred_dir='predictions', start_idx=0, fname_end=''):
    print('Testing...')
    fname = os.path.join(pred_dir, dataset_name, f'model_{nlp_embedding.name}_{nlp_classifier.name}{fname_end}.csv')
    predictions_all = []

    if start_idx == 0 and os.path.exists(fname):
        os.remove(fname)
    
    for i in range(start_idx, data_x.shape[0], batch_size):

        if i + batch_size > data_x.shape[0]:
            j = data_x.shape[0]
        else:
            j = i + batch_size
        
        print(f'Processing rows: {i} - {j - 1}')

        embeddings = nlp_embedding.embed_lyrics(data_x[i:j])
        predictions_enc = nlp_classifier.predict(embeddings)
        predictions = label_encoder.inverse_transform(predictions_enc)
        
        predictions_all.extend(predictions)

        pd.DataFrame(predictions.reshape(-1, 1)).to_csv(fname, mode='a', index=False, header=False)
    
    print('Success!')    
    
    return predictions_all

In [6]:
def get_results(y_true, y_pred):
    print('RESULTS:')
    print(f'accuracy = {metrics.accuracy_score(y_true=y_true, y_pred=y_pred)}')
    print(f'balanced accuracy = {metrics.balanced_accuracy_score(y_true=y_true, y_pred=y_pred)}')
    print(f'f1 score = {metrics.f1_score(y_true=y_true, y_pred=y_pred, average="weighted")}')

In [7]:
def train_and_save_results(emb, clf, x_train, y_train, x_test, y_test, dataset_name, le, batch_size=5000, epochs=1, fname_end=''):
    train(x_train, y_train, emb, clf, le, batch_size, dataset_name, epochs=epochs, fname_end=fname_end)
    y_pred = test(x_test, emb, clf, le, batch_size, dataset_name, fname_end=fname_end)
    get_results(y_test, y_pred)
    return

In [8]:
def add_normalized_lyrics(data):
    tokens = data.tokens.apply(literal_eval)
    data['normalized_lyrics'] = [' '.join(t) for t in tokens]

In [9]:
model_dir = os.path.join('models', dataset_name)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

pred_dir = os.path.join('predictions', dataset_name)
if not os.path.exists(pred_dir):
    os.makedirs(pred_dir)

In [10]:
train_data = pd.read_csv(f'data/train/{dataset_name}.csv')
test_data = pd.read_csv(f'data/test/{dataset_name}.csv')

In [11]:
train_data.genre.value_counts()

Rock       87661
Pop        33527
Metal      24294
Hip-Hop    21746
Country    16257
Name: genre, dtype: int64

In [12]:
test_data.genre.value_counts()

Rock       37569
Pop        14369
Metal      10412
Hip-Hop     9320
Country     6967
Name: genre, dtype: int64

In [13]:
add_normalized_lyrics(train_data)
add_normalized_lyrics(test_data)

In [14]:
genres = np.unique(train_data.genre)
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(genres)
label_encoder.classes_

array(['Country', 'Hip-Hop', 'Metal', 'Pop', 'Rock'], dtype=object)

In [15]:
indiv_genre_label = label_encoder.transform([indiv_genre])[0]

## Smaller BERT

In [ ]:
emb_sm_bert = SmallBert(max_words)

In [ ]:
clf_cnn = CNN(max_words * emb_sm_bert.embedding_size, len(genres), optimizer)
train_and_save_results(emb_sm_bert, clf_cnn_norm,
                       train_data.lyrics, train_data.genre, test_data.normalized_lyrics, test_data.genre, 
                       dataset_name, label_encoder, epochs=cnn_epochs, fname_end='_norm')

In [ ]:
clf_cnn_norm = CNN(max_words * emb_sm_bert.embedding_size, len(genres), optimizer)
train_and_save_results(emb_sm_bert, clf_cnn_norm,
                       train_data.normalized_lyrics, train_data.genre, test_data.normalized_lyrics, test_data.genre, 
                       dataset_name, label_encoder, epochs=cnn_epochs, fname_end='_norm')

## BERT

In [ ]:
emb_bert = Bert(max_words)

In [ ]:
clf_cnn_b = CNN(max_words * emb_bert.embedding_size, len(genres), optimizer)
train_and_save_results(emb_bert, clf_cnn_b,
                       train_data.lyrics, train_data.genre, test_data.lyrics, test_data.genre, 
                       dataset_name, label_encoder, epochs=cnn_epochs, batch_size=1000)

In [ ]:
clf_cnn_b_norm = CNN(max_words * emb_bert.embedding_size, len(genres), optimizer)
train_and_save_results(emb_bert, clf_cnn_b_norm,
                       train_data.normalized_lyrics, train_data.genre, test_data.normalized_lyrics, test_data.genre, 
                       dataset_name, label_encoder, epochs=cnn_epochs, batch_size=1000, fname_end='_norm')

## Large BERT

In [ ]:
emb_lr_bert = LargeBert(max_words)

In [ ]:
clf_cnn_bl = CNN(max_words * emb_lr_bert.embedding_size, len(genres), optimizer)
train_and_save_results(emb_lr_bert, clf_cnn_bl,
                       train_data.lyrics, train_data.genre, test_data.lyrics, test_data.genre, 
                       dataset_name, label_encoder, epochs=cnn_epochs, batch_size=1000)

In [ ]:
clf_cnn_bl_norm = CNN(max_words * emb_lr_bert.embedding_size, len(genres), optimizer)
train_and_save_results(emb_lr_bert, clf_cnn_bl_norm,
                       train_data.normalized_lyrics, train_data.genre, test_data.normalized_lyrics, test_data.genre, 
                       dataset_name, label_encoder, epochs=cnn_epochs, batch_size=1000, fname_end='_norm')

## GloVe

In [16]:
emb_glove = GloVe(max_words)

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


In [ ]:
clf_cnn_g = CNN(max_words * emb_glove.embedding_size, len(genres), optimizer)
train_and_save_results(emb_glove, clf_cnn_g, 
                       train_data.lyrics, train_data.genre, test_data.lyrics, test_data.genre, 
                       dataset_name, label_encoder, epochs=cnn_epochs)

In [ ]:
clf_cnn_g_norm = CNN(max_words * emb_glove.embedding_size, len(genres), optimizer)
train_and_save_results(emb_glove, clf_cnn_g_norm, 
                       train_data.normalized_lyrics, train_data.genre, test_data.normalized_lyrics, test_data.genre, 
                       dataset_name, label_encoder, epochs=cnn_epochs, fname_end='_norm')

### 2-step CNN

In [17]:
clf_cnn2_g_norm = CNN2Step(max_words * emb_glove.embedding_size, len(genres), optimizer, indiv_genre_label)
train_and_save_results(emb_glove, clf_cnn2_g_norm, 
                       train_data.normalized_lyrics, train_data.genre, test_data.normalized_lyrics, test_data.genre, 
                       dataset_name, label_encoder, epochs=cnn_epochs, fname_end='_norm')

Training...
Epoch: 1/5
Processing rows: 0 - 4999
81/81 [==============================] - 22s 274ms/step - loss: 1.5695 - accuracy: 0.5168
Processing rows: 5000 - 9999
82/82 [==============================] - 22s 271ms/step - loss: 1.0263 - accuracy: 0.5654
Processing rows: 10000 - 14999
83/83 [==============================] - 23s 277ms/step - loss: 0.9669 - accuracy: 0.6054
Processing rows: 15000 - 19999
83/83 [==============================] - 25s 296ms/step - loss: 0.9385 - accuracy: 0.6241
Processing rows: 20000 - 24999
82/82 [==============================] - 22s 269ms/step - loss: 0.8974 - accuracy: 0.6290
Processing rows: 25000 - 29999
81/81 [==============================] - 20s 250ms/step - loss: 0.8710 - accuracy: 0.6381
Processing rows: 30000 - 34999
83/83 [==============================] - 21s 252ms/step - loss: 0.8583 - accuracy: 0.6579
Processing rows: 35000 - 39999
83/83 [==============================] - 22s 265ms/step - loss: 0.8441 - accuracy: 0.6438
Processing rows:

82/82 [==============================] - 23s 275ms/step - loss: 0.6798 - accuracy: 0.7347
Processing rows: 10000 - 14999
83/83 [==============================] - 23s 276ms/step - loss: 0.6940 - accuracy: 0.7246
Processing rows: 15000 - 19999
83/83 [==============================] - 23s 280ms/step - loss: 0.6744 - accuracy: 0.7323
Processing rows: 20000 - 24999
82/82 [==============================] - 23s 280ms/step - loss: 0.6478 - accuracy: 0.7320
Processing rows: 25000 - 29999
81/81 [==============================] - 21s 258ms/step - loss: 0.6316 - accuracy: 0.7486
Processing rows: 30000 - 34999
83/83 [==============================] - 21s 256ms/step - loss: 0.6164 - accuracy: 0.7562
Processing rows: 35000 - 39999
83/83 [==============================] - 22s 269ms/step - loss: 0.6145 - accuracy: 0.7619
Processing rows: 40000 - 44999
84/84 [==============================] - 23s 277ms/step - loss: 0.6179 - accuracy: 0.7554
Processing rows: 45000 - 49999
82/82 [=========================

83/83 [==============================] - 23s 275ms/step - loss: 0.5014 - accuracy: 0.8017
Processing rows: 15000 - 19999
83/83 [==============================] - 23s 275ms/step - loss: 0.4671 - accuracy: 0.8206
Processing rows: 20000 - 24999
82/82 [==============================] - 23s 278ms/step - loss: 0.4210 - accuracy: 0.8435
Processing rows: 25000 - 29999
81/81 [==============================] - 20s 252ms/step - loss: 0.4141 - accuracy: 0.8353
Processing rows: 30000 - 34999
83/83 [==============================] - 21s 251ms/step - loss: 0.4238 - accuracy: 0.8362
Processing rows: 35000 - 39999
83/83 [==============================] - 22s 267ms/step - loss: 0.4003 - accuracy: 0.8460
Processing rows: 40000 - 44999
84/84 [==============================] - 23s 279ms/step - loss: 0.3870 - accuracy: 0.8562
Processing rows: 45000 - 49999
82/82 [==============================] - 23s 275ms/step - loss: 0.3708 - accuracy: 0.8544
Processing rows: 50000 - 54999
83/83 [=========================

83/83 [==============================] - 23s 276ms/step - loss: 0.2549 - accuracy: 0.9010
Processing rows: 20000 - 24999
82/82 [==============================] - 22s 274ms/step - loss: 0.2435 - accuracy: 0.9043
Processing rows: 25000 - 29999
81/81 [==============================] - 20s 252ms/step - loss: 0.2190 - accuracy: 0.9176
Processing rows: 30000 - 34999
83/83 [==============================] - 21s 250ms/step - loss: 0.2480 - accuracy: 0.9113
Processing rows: 35000 - 39999
83/83 [==============================] - 22s 268ms/step - loss: 0.2360 - accuracy: 0.9147
Processing rows: 40000 - 44999
84/84 [==============================] - 23s 280ms/step - loss: 0.2234 - accuracy: 0.9175
Processing rows: 45000 - 49999
82/82 [==============================] - 23s 278ms/step - loss: 0.2007 - accuracy: 0.9272
Processing rows: 50000 - 54999
83/83 [==============================] - 23s 274ms/step - loss: 0.2126 - accuracy: 0.9148
Processing rows: 55000 - 59999
81/81 [=========================

82/82 [==============================] - 23s 275ms/step - loss: 0.1463 - accuracy: 0.9450
Processing rows: 25000 - 29999
81/81 [==============================] - 20s 252ms/step - loss: 0.1436 - accuracy: 0.9482
Processing rows: 30000 - 34999
83/83 [==============================] - 21s 251ms/step - loss: 0.1615 - accuracy: 0.9367
Processing rows: 35000 - 39999
83/83 [==============================] - 22s 268ms/step - loss: 0.1712 - accuracy: 0.9426
Processing rows: 40000 - 44999
84/84 [==============================] - 23s 277ms/step - loss: 0.1603 - accuracy: 0.9459
Processing rows: 45000 - 49999
82/82 [==============================] - 23s 280ms/step - loss: 0.1223 - accuracy: 0.9544
Processing rows: 50000 - 54999
83/83 [==============================] - 23s 276ms/step - loss: 0.1338 - accuracy: 0.9517
Processing rows: 55000 - 59999
81/81 [==============================] - 22s 274ms/step - loss: 0.1845 - accuracy: 0.9352
Processing rows: 60000 - 64999
81/81 [=========================

88/88 [==============================] - 5s 53ms/step
[0 1 2 3 4]
[0 1]
[0 1 2 3]
Processing rows: 35000 - 39999
88/88 [==============================] - 4s 47ms/step
[0 1 2 3 4]
[0 1]
[0 1 2 3]
Processing rows: 40000 - 44999
86/86 [==============================] - 4s 52ms/step
[0 1 2 3 4]
[0 1]
[0 1 2 3]
Processing rows: 45000 - 49999
87/87 [==============================] - 5s 52ms/step
[0 1 2 3 4]
[0 1]
[0 1 2 3]
Processing rows: 50000 - 54999
87/87 [==============================] - 5s 52ms/step
[0 1 2 3 4]
[0 1]
[0 1 2 3]
Processing rows: 55000 - 59999
89/89 [==============================] - 4s 47ms/step
[0 1 2 3 4]
[0 1]
[0 1 2 3]
Processing rows: 60000 - 64999
88/88 [==============================] - 4s 47ms/step
[0 1 2 3 4]
[0 1]
[0 1 2 3]
Processing rows: 65000 - 69999
86/86 [==============================] - 4s 51ms/step
[0 1 2 3 4]
[0 1]
[0 1 2 3]
Processing rows: 70000 - 74999
88/88 [==============================] - 5s 51ms/step
[0 1 2 3 4]
[0 1]
[0 1 2 3]
Processing row

## Word2vec

In [ ]:
emb_wv = Word2vec(max_words)

In [ ]:
clf_cnn_w_nn = CNN(max_words * emb_wv.embedding_size, len(genres), 'adam')
train_and_save_results(emb_wv, clf_cnn_w_nn, 
                       train_data.lyrics, train_data.genre, test_data.lyrics, test_data.genre, 
                       dataset_name, label_encoder, epochs=cnn_epochs)

In [ ]:
clf_cnn_w = CNN(max_words * emb_wv.embedding_size, len(genres), 'adam')
train_and_save_results(emb_wv, clf_cnn_w, 
                       train_data.normalized_lyrics, train_data.genre, test_data.normalized_lyrics, test_data.genre, 
                       dataset_name, label_encoder, epochs=cnn_epochs, fname_end='_norm')